<a href="https://colab.research.google.com/github/bioinfo-gao/LLM/blob/main/PyTorch/4_buildmodel_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# For tips on running notebooks in Google Colab, see
# https://docs.pytorch.org/tutorials/beginner/colab
%matplotlib inline

[Learn the Basics](intro.html) \|\|
[Quickstart](quickstart_tutorial.html) \|\|
[Tensors](tensorqs_tutorial.html) \|\| [Datasets &
DataLoaders](data_tutorial.html) \|\|
[Transforms](transforms_tutorial.html) \|\| **Build Model** \|\|
[Autograd](autogradqs_tutorial.html) \|\|
[Optimization](optimization_tutorial.html) \|\| [Save & Load
Model](saveloadrun_tutorial.html)

Build the Neural Network
========================

Neural networks comprise of layers/modules that perform operations on
data. The [torch.nn](https://pytorch.org/docs/stable/nn.html) namespace
provides all the building blocks you need to build your own neural
network. Every module in PyTorch subclasses the
[nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html).
A neural network is a module itself that consists of other modules
(layers). This nested structure allows for building and managing complex
architectures easily.

In the following sections, we\'ll build a neural network to classify
images in the FashionMNIST dataset.


In [3]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

<font color="red"> Get Device for Training
=======================

We want to be able to train our model on an
[accelerator](https://pytorch.org/docs/stable/torch.html#accelerators)
such as <font color=" blue"> CUDA, MPS, MTIA, or XPU. </font> If the current accelerator is
available, we will use it. Otherwise, we use the CPU.


In [4]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


Define the Class
================

We define our neural network by <font color=" blue">subclassing `nn.Module` </font>, and initialize
the neural network layers in `__init__`. <font color=" red"> Every `nn.Module` subclass
</font>implements the operations on input data in <font color=" green"> the `forward` method. </font>


In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x): # Every nn.Module subclass implements forward method for input data
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

We create an instance of `NeuralNetwork`, and move it to the <font color=" red"> `device` </font>,
and print its structure.


In [6]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


To use the model, we pass it the input data. This executes the model\'s
`forward`, along with some [background
operations](https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866).
<font color=" red"> Do not call `model.forward()` directly!

Calling the model on the input returns a 2-dimensional tensor with dim=0
corresponding to each output of 10 raw predicted values for each class,
and dim=1 corresponding to the individual values of each output. We get
the prediction probabilities by passing it through an instance of the
`nn.Softmax` module.


这段话描述的模型输出是一个二维张量（2D Tensor），其形状通常表示为 (批次大小 $\times$ 类别数)。


1. dim=0 (行): 批次维度

对应：dim=0 对应于每个输出，即批次 (Batch) 中的每个独立样本/图片。  

示例： 如果 dim=0 的大小是 64，表示您同时向模型输入了 64 张图片或数据样本。  

2. dim=1 (列): 类别维度

对应： dim=1 对应于每个输出的个体值，即每个类别对应的原始预测值。  

示例： 文中提到 10 个原始预测值，意味着这是一个**10 分类问题**。对于批次中的每个样本，模型都会给出一个长度为 10 的原始分数向量。

原始值性质： 这些原始值被称为 Logits（对数几率），它们是无界的（可以是负数、零或正数），并且不求和为 1。



为了将这些原始分数 $\text{Logits}$ 转化为概率 ($\text{Probabilities}$)，您需要使用 $\text{nn.Softmax}$ 模块。

1. 转换目的$\text{Softmax}$ 函数执行归一化：概率化： 将 $\text{Logits}$ 映射到 $\text{[0, 1]}$ 的区间。总和为 1： 确保同一维度（即所有类别）的概率总和为 $\text{1}$。这使得您可以将输出解释为模型对每个类别的置信度。

2. 数学基础$\text{Softmax}$ 函数通过指数化并除以总和来实现归一化。对于输出向量 $\mathbf{z}$ 中的第 $i$ 个元素 $z_i$，其 $\text{Softmax}$ 结果 $p_i$ 为：$$p_i = \frac{e^{z_i}}{\sum_{j=1}^{K} e^{z_j}}$$

3. 应用维度 (Crucial Dimension)当您在 $\text{PyTorch}$ 中使用 $\text{nn.Softmax}$ 时，您必须指定作用的维度。正确做法： $\text{Softmax}$ 必须作用于 $\text{dim}=1$（即类别维度）。


In [7]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X) # model output ！ <<<====================================

pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([3], device='cuda:0')


------------------------------------------------------------------------


Model Layers
============

Let\'s break down the layers in the FashionMNIST model. To illustrate
it, we will take a sample minibatch of <font color=" red">  3 images of size 28x28 </font> and see
what happens to it as we pass it through the network.


In [8]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


nn.Flatten
==========

We initialize the
[nn.Flatten](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html)
layer to convert each 2D 28x28 image into a contiguous array of 784
pixel values ( the minibatch dimension (at dim=0) is maintained).


In [9]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


nn.Linear
=========

The [linear
layer](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)
is a module that applies a linear transformation on the input using its
stored weights and biases.


In [10]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


nn.ReLU
=======

Non-linear activations are what create the complex mappings between the
model\'s inputs and outputs. They are applied after linear
transformations to introduce *nonlinearity*, helping neural networks
learn a wide variety of phenomena.

In this model, we use
[nn.ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html)
between our linear layers, but there\'s other activations to introduce
non-linearity in your model.


In [11]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.1275,  0.1048, -0.4153,  0.7854,  0.2479,  0.0022, -0.0528,  0.2747,
          0.0106, -0.7177, -0.5073, -0.2946,  0.3168, -0.1310,  0.4564, -0.5561,
          0.1361, -0.0796,  0.0641,  0.2655],
        [ 0.1759,  0.4547, -0.8307,  0.4997,  0.0839, -0.2091, -0.0277,  0.0675,
          0.0167, -0.4265, -0.5054,  0.1311,  0.4294, -0.5734,  0.4059, -0.4835,
          0.1121, -0.2282, -0.0095,  0.2411],
        [-0.2416,  0.1318, -0.5253,  0.1219,  0.3698, -0.2199, -0.1411,  0.1202,
         -0.3719, -0.6189, -0.1769,  0.0453, -0.2176, -0.3937,  0.1943, -0.5890,
         -0.1260, -0.1086, -0.0094,  0.2459]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.1275, 0.1048, 0.0000, 0.7854, 0.2479, 0.0022, 0.0000, 0.2747, 0.0106,
         0.0000, 0.0000, 0.0000, 0.3168, 0.0000, 0.4564, 0.0000, 0.1361, 0.0000,
         0.0641, 0.2655],
        [0.1759, 0.4547, 0.0000, 0.4997, 0.0839, 0.0000, 0.0000, 0.0675, 0.0167,
         0.0000, 0.0000, 0.1311, 0.4294, 0.0000, 0.40

nn.Sequential
=============

[nn.Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html)
is an ordered container of modules. The data is passed through all the
modules in the same order as defined. You can use sequential containers
to put together a quick network like `seq_modules`.

$\text{nn.Sequential}$ 是一种组织神经网络层的方式，它把一系列操作（模块）按顺序打包成一个单一的模块，让数据可以像流水线一样通过它们。

🏗️ nn.Sequential 解释

1. 有序容器 (Ordered Container)概念： $\text{nn.Sequential}$ 就像一个列表或数组，它按照您定义的顺序严格地容纳多个独立的网络层（模块，如 $\text{nn.Linear}$、$\text{nn.Conv2d}$、$\text{nn.ReLU}$ 等）。作用： 它的主要目的是结构化和简化神经网络的构建过程。

2. 数据流 (Data Flow)机制： 数据是按照定义的相同顺序依次通过所有模块的。过程：输入数据进入 $\text{Sequential}$ 容器中的 第一个模块。第一个模块的输出作为 第二个模块 的输入。这个过程重复进行，直到数据通过容器中的所有模块。简洁性： 它将原来需要多行代码定义的前向传播 ($\text{forward}$) 逻辑（例如 x = layer1(x); x = layer2(x); x = activation(x)）简化为一行代码：output = seq_modules(input)。

3. 构建快速网络 (Quick Network)示例： 就像您提到的 seq_modules，$\text{nn.Sequential}$ 最常用于构建简单、前馈（$\text{Feed-forward}$）结构的网络。代码示例：

In [ ]:
# sequential 示例， 我个人的记忆这就是Tensorflow 和 PyTorch的区别
import torch.nn as nn

# 传统方式需要自己定义 forward()
# class SimpleNet(nn.Module):
#     def __init__(self):
#         self.layer1 = nn.Linear(784, 128)
#         self.relu = nn.ReLU()
#         self.layer2 = nn.Linear(128, 10)
#     def forward(self, x):
#         x = self.layer1(x)
#         x = self.relu(x)
#         x = self.layer2(x)
#         return x

# 使用 nn.Sequential (更简洁)
seq_modules = nn.Sequential(
    nn.Linear(784, 128),  # 模块 1
    nn.ReLU(),            # 模块 2
    nn.Linear(128, 10)    # 模块 3
)

# 数据输入：output = seq_modules(input)

In [13]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

nn.Softmax
==========

The last linear layer of the neural network returns [logits]{.title-ref}
- raw values in \[-infty, infty\] - which are passed to the
[nn.Softmax](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html)
module. The logits are scaled to values \[0, 1\] representing the
model\'s predicted probabilities for each class. `dim` parameter
indicates the dimension along which the values must sum to 1.


In [14]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

Model Parameters
================

Many layers inside a neural network are *parameterized*, i.e. have
associated weights and biases that are optimized during training.
Subclassing `nn.Module` automatically tracks all fields defined inside
your model object, and makes all parameters accessible using your
model\'s `parameters()` or `named_parameters()` methods.

In this example, we iterate over each parameter, and print its size and
a preview of its values.


In [15]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0178, -0.0316,  0.0056,  ...,  0.0188,  0.0069, -0.0246],
        [ 0.0121, -0.0231, -0.0181,  ...,  0.0172, -0.0071, -0.0151]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0210, -0.0172], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0294, -0.0356,  0.0176,  ..., -0.0117, -0.0129,  0.0258],
        [-0.0267,  0.0015, -0.0097,  ...,  0.0387, -0.0131, -0.0306]],
       device='cuda:0', grad_fn=<Sl

------------------------------------------------------------------------


Further Reading
===============

-   [torch.nn API](https://pytorch.org/docs/stable/nn.html)
